In [1]:
import glob

#Set images directory path here
#make sure path ends with a '/'
images_dir = '/media/golden/72FFC6EE48B5CF39/kenya-tracking/raw_frames/test_extract/extracts/'


file_list = glob.glob(images_dir + '**/*.jpg', recursive=True)
if not file_list:
    print('No files found')
else:
    print(len(file_list), ' files found')
    example_split_file = file_list[0].split('.')
    #multiple files per frame -> extracted object images
    if len(example_split_file) >= 3:
        print('These are extracted images from frame -> Two level sort')
        file_list.sort(key=lambda file: (int(file.split('.')[0].split('_')[-1]), int(file.split('.')[2])))
    else:
        raise ValueError('This image files are in an unexpected format')


            

12433  files found
These are extracted images from frame -> Two level sort


In [2]:
import numpy as np
head_boxes_file = '/media/golden/72FFC6EE48B5CF39/kenya-tracking/head-annotations/test_extract/boxes.npy'
head_boxes = np.load(head_boxes_file)

In [3]:
head_boxes.shape

(12433,)

In [4]:
def get_head_position(head_box, frame_width, frame_height):
    head_center = np.ones(2)

    #first get center of the head_box
    head_center[0] = frame_height * (head_box[0] + (head_box[2] - head_box[0]) / 2) 
    head_center[1] = frame_width * (head_box[1] + (head_box[3] - head_box[1]) / 2)
     
    return head_center

In [5]:
frame_size = 160
head_positions = []
for head_box in head_boxes:
    try:
        head_positions.append(get_head_position(head_box[0], frame_size, frame_size))
    except:
        head_positions.append(None)


In [6]:
from PIL import Image, ImageDraw
#Test some outputs
max_image = 1000
step = 50
for image_num, image_file in enumerate(file_list[:max_image:step]):
    image_index = image_num * step
    image = Image.open(image_file)
    if head_positions[image_index] is not None:
        draw = ImageDraw.Draw(image)
        head_vector = np.ones(2)
        head_vector[0] = head_positions[image_index][0] - frame_size / 2
        head_vector[1] = head_positions[image_index][1] - frame_size / 2
        draw.line((head_positions[image_index][1], head_positions[image_index][0], 
                   head_positions[image_index][1] + head_vector[1], head_positions[image_index][0] + head_vector[0]), 
                  fill=(128,30,128), width=5)
    image.show()

In [7]:
import cv2

In [8]:
image = cv2.imread(file_list[12])

In [ ]:
cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()